## Imports 

In [1]:
# TODO 
# the dates might mess up if it's a single digit day of 
# the month since I just take the first 5 characters of the date string at fivethirtyeight

In [2]:
# Imports 
import pandas as pd 
import requests
from bs4 import BeautifulSoup 
import pandas as pd 
import re
import time    

## Helper Function 

In [3]:
# Convert money lines to probabilites 
def to_prob(money_line):
    if money_line < 0: 
        prob = -money_line / (-money_line + 100) 
    elif money_line > 0: 
        prob = 100 / (money_line + 100) 
    else: 
        prob = None 

    return (prob) 

## Scrape 538

In [10]:
# Pull URL and page content 
url = 'https://projects.fivethirtyeight.com/2019-mlb-predictions/games/'
page_response = requests.get(url)
page_content = BeautifulSoup(page_response.text, 'html.parser')

In [20]:
rows = page_content.findAll(class_="tr")

dict_list = []

for row in rows:
    # Error handling 
    try:
        date = row.find(class_='td td-datetime').getText()
    except:
        score = None
    long_name = row.find(class_='team-name long').getText()
    short_name = row.find(class_='team-name short').getText()
    pitcher = row.find(class_='pitcher-name').getText()
    num_rating = row.find(class_='td number td-number rating-adj').getText().replace('=', '')
    win_prob = row.find(class_='td number td-number win-prob').getText()
    
    # Error handling 
    try:
        score = row.find(class_='td number td-number score').getText()
    except:
        score = None
    
    d = {
        #'date': date[:4], # Hackier way of pulling date
        'date': re.search(r'[0-9]+/[0-9]+', date).group(), # Regex pulls date from string 
        'long_name': long_name,
        'short_name': short_name,
        'pitcher': pitcher,
        'num_rating': num_rating, 
        'win_prob': win_prob.replace('%', ''),
        'score': score
    }
    
    dict_list.append(d)

# Create fivethirtyeight dataframe
fte_df = pd.DataFrame(dict_list)
fte_df

,date,long_name,num_rating,pitcher,score,short_name,win_prob
0,7/21,Red Sox,1532,Cashner,,BOS,66
1,7/21,Orioles,1415,Wojciechowski,,BAL,34
2,7/21,Rockies,1516,Marquez,,COL,36
3,7/21,Yankees,1616,Paxton,,NYY,64
4,7/21,Cardinals,1531,Flaherty,,STL,52
5,7/21,Reds,1519,DeSclafani,,CIN,48
6,7/21,Royals,1443,Sparkman,,KC,37
7,7/21,Indians,1539,Plesac,,CLE,63
8,7/21,Blue Jays,1466,Waguespack,,TOR,55
9,7/21,Tigers,1435,Alexander,,DET,45


## Scrape ESPN

In [6]:
# Get Page and Content 
url = 'http://www.espn.com/mlb/lines'
page_response = requests.get(url)
page_content = BeautifulSoup(page_response.text, 'html.parser')

# Pull Rows of Data
rows = page_content.findAll(class_="oddrow")
rows = rows + (page_content.findAll(class_="evenrow"))

In [7]:
# List Precursor 
dict_list = []

# Main Text Parsing of MLB Games 
for row in rows: 
    if row.findAll("td")[0].getText() == 'Westgate':
        bookee = (row.findAll("td")[0].getText())

        row = (str(row.findAll("td")[1]).replace("<td style=\"text-align:center;\">", "").replace("</td>","").split("<br/>"))
        
        if (row[0] == 'N/A'): # Some rows are blank 
            pass
        else:
            team_1 = row[0].split(":")[0]
            team_2 = row[1].split(":")[0]
            moneyline_1 = row[0].split(":")[1].strip()
            moneyline_2 = row[1].split(":")[1].strip()    
        
        #print(bookee, team_1, moneyline_1, team_2, moneyline_2)
        
        # Create dictionaries to add data to df 
        d1 = {
        'bookee': bookee,
        'short_name': team_1,
        'moneyline': moneyline_1
        }
        
        d2 = {
        'bookee': bookee,
        'short_name': team_2,
        'moneyline': moneyline_2    
        }
         
        dict_list.append(d1)
        dict_list.append(d2)
        
        
# Create df and Features 
mlb_df = pd.DataFrame(dict_list)   
mlb_df['prob'] = [round(to_prob(int(i))*100) for i in mlb_df['moneyline']]
mlb_df['date'] = str(time.strftime("%-m/%d")) #removes leading 0   
mlb_df

,bookee,moneyline,short_name,prob,date
0,Westgate,180,COL,36,7/21
1,Westgate,-195,NYY,66,7/21
2,Westgate,-210,BOS,68,7/21
3,Westgate,190,BAL,34,7/21
4,Westgate,230,CHW,30,7/21
5,Westgate,-260,TB,72,7/21
6,Westgate,200,KC,33,7/21
7,Westgate,-220,CLE,69,7/21
8,Westgate,200,KC,33,7/21
9,Westgate,-220,CLE,69,7/21


## Compare Probabilities

In [21]:
# Join on team, date 
result = pd.merge(fte_df,
                 mlb_df,
                 on=['date', 'short_name'])

# Find differences
result['difference'] = result['win_prob'].astype('int64') - result['prob'].astype('int64')
result

# Sort and output
result.drop_duplicates(inplace = True)
result.sort_values(by='difference', ascending=False)

,date,long_name,num_rating,pitcher,score,short_name,win_prob,bookee,moneyline,prob,difference
27,7/21,Mariners,1477,Kikuchi,,SEA,48,Westgate,160,38,10
23,7/21,Diamondbacks,1539,Young,,ARI,49,Westgate,135,43,6
8,7/21,Royals,1443,Sparkman,,KC,37,Westgate,200,33,4
12,7/21,White Sox,1453,Cease,,CHW,34,Westgate,230,30,4
16,7/21,Athletics,1539,Mengden,,OAK,47,Westgate,125,44,3
4,7/21,Cardinals,1531,Flaherty,,STL,52,Westgate,-105,51,1
24,7/21,Marlins,1454,Yamamoto,,MIA,27,Westgate,280,26,1
2,7/21,Rockies,1516,Marquez,,COL,36,Westgate,180,36,0
14,7/21,Rangers,1512,Lynn,,TEX,43,Westgate,130,43,0
1,7/21,Orioles,1415,Wojciechowski,,BAL,34,Westgate,190,34,0


## Store Results 